# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
# import csv
csv_path = "output_data/cities.csv"
# read csv
weather_df = pd.read_csv(csv_path)
weather_df


,Unnamed: 0,city,temp_max,humidity,clouds,windspeed,lat,lng,country,date
0,0,kalmunai,76.19,89,33,9.57,7.4167,81.8167,LK,1612747621
1,1,altay,21.06,81,66,3.49,47.8667,88.1167,CN,1612747621
2,2,chokurdakh,-15.34,88,100,7.94,70.6333,147.9167,RU,1612747373
3,3,qaanaaq,15.80,85,100,6.40,77.4840,-69.3632,GL,1612747365
4,4,morris,30.99,78,70,1.99,40.8334,-74.5329,US,1612747621
5,5,yuncheng,38.17,68,0,5.95,35.0231,110.9928,CN,1612747621
6,6,trairi,79.84,81,0,8.43,-3.2778,-39.2689,BR,1612747383
7,7,castro,58.23,67,0,7.05,-24.7911,-50.0119,BR,1612747515
8,8,lavrentiya,-8.16,86,0,8.41,65.5833,-171.0000,RU,1612747561
9,9,kitob,44.60,76,97,4.61,39.0843,66.8332,UZ,1612747622


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["lat", "lng"]].astype(float)

# Convert humidity to float and store
humidity = weather_df["humidity"].astype(float)

In [16]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
hotel_df = weather_df.loc[weather_df['temp_max']> 70].loc[weather_df['temp_max']<80] \
.loc[weather_df['windspeed']< 10].loc[weather_df['clouds']== 0]

hotel_df

,Unnamed: 0,city,temp_max,humidity,clouds,windspeed,lat,lng,country,date
6,6,trairi,79.84,81,0,8.43,-3.2778,-39.2689,BR,1612747383


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# find the closest hotel for each set of coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat, lon from df
    lat = row["lat"]
    long = row["lng"]
    
    # add keyword to params dict
    params['location'] = f"{lat},{long}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['city']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 6: trairi.
Closest hotel is janelas do mar.
------------


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Display figure
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig




NameError: name 'hotel_info' is not defined

In [27]:
plt.savefig('hotel_heatmap.png')


<Figure size 432x288 with 0 Axes>